#  Exploring, Visualizing, and Plotting BioSCape Field and AVIRIS-NG Data
## BioSCape Data Skills Workshop: From the Field to the Image
![Bioscape](images/121229-87.png)

[**BioSCape**](https://www.bioscape.io/), the Biodiversity Survey of the Cape, is NASA’s first biodiversity-focused airborne and field campaign that was conducted in South Africa in 2023. BioSCape’s primary objective is to study the structure, function, and composition of the region’s ecosystems, and how and why they are changing. 

BioSCape's airborne dataset is unprecedented, with `AVIRIS-NG`, `PRISM`, and `HyTES` imaging spectrometers capturing spectral data across the UV, visible and infrared at high resolution and `LVIS` acquiring coincident full-waveform lidar. BioSCape's `field dataset` is equally impressive, with 18 PI-led projects collecting data ranging from the diversity and phylogeny of plants, kelp and phytoplankton, eDNA, landscape acoustics, plant traits, blue carbon accounting, and more

This workshop will equip participants with the skills to find, subset, and visualize the various BioSCape field and airborne (imaging spectroscopy and full-waveform lidar) data sets. Participants will learn data skills through worked examples in terrestrial and aquatic ecosystems, including: wrangling lidar data, performing band math calculations, calculating spectral diversity metrics, machine learning and image classification, and mapping functional traits using partial least squares regression. The workshop format is a mix of expert talks and interactive coding notebooks and will be run through the BioSCape Cloud computing environment.

**Date:** October 9 - 11, 2024 Cape Town, South Africa</center>

**Host:** NASA’s Oak Ridge National Laboratory Distributed Active Archive Center (ORNL DAAC), in close collaboration with BioSCape, the South African Environmental Observation Network (SAEON), the University of Wisconsin Madison (Phil Townsend), The Nature Conservancy (Glenn Moncrieff), the University of California Merced (Erin Hestir), the University of Cape Town (Jasper Slingsby), Jet Propulsion Laboratory (Kerry Cawse-Nicholson), and UNESCO.

**Instructors:** 
- In-person contributors: Anabelle Cardoso, Erin Hestir, Phil Townsend, Henry Frye, Glenn Moncrieff, Jasper Slingsby, Michele Thornton, Rupesh Shrestha
- Virtual contributors: Kerry Cawse-Nicholson, Nico Stork, Kyle Kovach

**Audience:** This training is primarily intended for government natural resource management agency representatives and field technicians in South Africa, as well as local academics and students, especially those connected to the BioSCape Team. 

## Overview 
This tutorial will explore BioSCape Campaign `airborne data` currently available on the BioSCape SMCE S3 storage as well as preliminary `vegetation plot data`. Participants will learn to access cloud-based storage and explore files.  Analysis will occur with combinations of data available on S3 storage as well as vector plot and polygon data brought to the Hub. Demonstrations will provide methods to locate AVIRIS-NG scenes associated with vegetation plot data.  Vegetation spectral profiles will be extracted and graphed from projected plot locations.   

### Learning Objectives
1.  Mount BioSCape SMCE S3 BioSCape object storage and explore file content
2.  Using BioSCape preliminary field data, model bringing your own data to a cloud-hub environment for visualization and analysis.
3.  Visualize, plot, and subset AVIRIS-NG scene identify based on vegetation plot locations.
4.  Examine and visualize AVIRIS-NG quick look and L2 Reflectance data using both GDAL and python libraries for raster analysis.
6.  Project plot locations from geographic to local projection coordinate reference system.
7.  Extract and graph vegetation plot-level spectral profiles of AVIRIS-NG data.



### Load Python Modules

In [ ]:
import s3fs
import rioxarray
from os import path
import matplotlib.pyplot as plt
import rasterio
from os import path
import geopandas as gpd
import xarray as xr
import folium
from osgeo import gdal
import numpy as np
from pyproj import Proj
gdal.UseExceptions()

## Explore the BioSCape SMCE S3 data holdings

Let's start by exploring the BioSCape Airborne data currently available on the cloud in Amazon Storage. This AWS S3 storage is specific to the SMCE that provides data access and analytics environment to the BioSCape Science Team as well as interested researchers.  
We'll learn how to mount the S3 object storage on our local environment, as well as how to bring other data to the analysis.

- **SMCE** = Science Managed Cloud Environment
- **S3** = Amazon Simple Storage Service (S3) is a cloud storage service that allows users to store and retrieve data
- **S3Fs** is a `Pythonic` open source tool that mounts S3 object storage locally.  S3Fs provides a filesystem-like interface for accessing objects on S3.
    - The top-level class **`S3FileSystem`** holds connection information and allows typical file-system style operations like `ls`
    - `ls` is a UNIX command to list computer files and directories

In [ ]:
# Use S3Fs to list the BioSCape data on the BioSCape SMCE S3 storaage
s3 = s3fs.S3FileSystem(anon=False)
files = s3.ls('bioscape-data/')
files

## BioSCape Flight Data

The BioSCape Campaign (Oct - Nov, 2023) flew 4 airborne instruments on 2 aircraft. 

- `Gulfstream 3`: **AVIRIS-Next Generation** and **PRISM**
- `Gulfstream 5`: **HyTES** and **LVIS**

The NASA Jet Propulsion Laboratory provides the **BioSCape Data Portal** (https://popo.jpl.nasa.gov/mmgis-aviris/?mission=BIOSCAPE) which shows flight line visualization, quick look images, and BioSCape Team Regions of Interest (ROIs).  Download access to preliminary airborne data is available.


### Let's look deeper into each airborne folder on the SMCE
### **AVIRIS-Next Generation (AVNG)**
- We'll spend a little more time looking into the AVIRIS-NG files as these data are a focus of this Notebook


In [ ]:
AVNG_flightlines = s3.ls('bioscape-data/AVNG/')
AVNG_flightlines[:10]

In [ ]:
AVNGfl_count = len(AVNG_flightlines)
AVNGfl_count

In [ ]:
# looking into the ang20231022t092801 folder
AVNG_scenes = s3.ls('bioscape-data/AVNG/ang20231022t092801')
AVNG_scenes

In [ ]:
# Explore the AVNG folder that holds an AVNG scene's data
AVNG_scene_data = s3.ls('bioscape-data/AVNG/ang20231022t092801/ang20231022t092801_000')
AVNG_scene_data

### File **Naming Conventions** provide information about each flight line, scene, and data type.

- **`L1B`** are AVIRIS-NG Surface Radiance
- **`L2A`** are AVIRIS-NG Surface Reflectance

| Dataset | Description | 
| -------- | --- |
| *_RFL_ORT_QL.tif | Reflectance Quick Look Image (3 band) |
| *-RFL_ORT | Reflectance ENVI binary file (425 band)|
| *_RFL_ORT.hdr | Reflectance ENVI header file (txt file)|

<center><img src="images/ANG_naming.jpg"/></center>

### **Portable Remote Imaging Spectrometer (PRISM)**

In [ ]:
PRISM_flightlines = s3.ls('bioscape-data/PRISM')
PRISM_flightlines

#### There are Level 2 (L2) for PRISM data in the SCME
- ENVI file format in binary/header pairs


In [ ]:
PRISM_flightlines = s3.ls('bioscape-data/PRISM/L2')
PRISM_flightlines[:10]

- the PRISM are available in ENVI file formats as binary/header pairs

### **Land, Vegetation, and Ice Sensor (LVIS)**

In [ ]:
LVIS_flightlines = s3.ls('bioscape-data/LVIS/')
LVIS_flightlines

#### Let's look into the LVIS Level 2 (L2) data

In [ ]:
LVIS_flightlines_L2 = s3.ls('bioscape-data/LVIS/L2')
LVIS_flightlines_L2[:10]

- LVIS L2 data are ASCII Text files

### **Hyperspectral Thermal Emissions Spectrometer (HyTES)**
- BioSCape HyTES data is currently available from [**NASA JPL HyTES**](https://hytes.jpl.nasa.gov/order!) distribution site

## Examine Field Data Sites: Vegetation Site Data 

### Bringing your own data to the Cloud
We know that researchers will often want to bring their own data to the cloud.  You might have files necessary for your analysis, but they are on your local machine.  

For example, you might have field data in vector formats (point, polygon) that you have as local files on your personal device. 

On the BioSCape SMCE Hub, you can easily upload those files into your working area.  Simply click the `Upload Files` button at the top of the folder/file listing panel.  You can also drag and drop folders into the file listing panel.  

![upload_button](images/upload_button.PNG)

For the purpose of this Notebook, we're using a polygon shape file of Vegetation Polygons as well as point locations of the Vegetation Polygon Center for a few specific polygons in the Cape Peninsula.

- Vegetation Polygons (**BioSCapeVegPolys2024_09_12.shp**) 
- Vegetation Polygon Centers (**Focal_Veg_Center_Plots.geojson**) 






**`Geopandas`** is an open source project to make working with geospatial data in python easier. GeoPandas extends the datatypes used by pandas to allow spatial operations on geometric types.

In [ ]:
# read the BioSCapeVegPolys shape file and convert to geojson using geopandas

veg_polys_gdf = gpd.read_file('data/BioSCapeVegPolys2024_09_12.shp')
veg_polys_gdf.to_file('veg_polys_json.geojson', driver='GeoJSON')
veg_polys_json_gdf = gpd.read_file('veg_polys_json.geojson')

vegpolys_cen_new = gpd.read_file('data/Focal_Veg_Center_Plots.geojson')

**^^^ You should see a new geojson files in your folder area**^^^

In [ ]:
# Take a look at the data in the Focal_Veg_Center_Plots file
vegpolys_cen_new

In [ ]:
# explore the BioSCapeVegPolys data
list(veg_polys_gdf.columns)

In [ ]:
veg_polys_gdf

In [ ]:
# examine the coordinate reference system of the polygon file
veg_polys_gdf.crs

## Visualize data
### **`Folium`** is a powerful Python library that creates interactive Leaflet maps. It allows visualization of data in Python on an interactive map

- Overlay geographic data like country borders or city boundaries from GeoJSON or TopoJSON files.
- Folium integrates seamlessly with Jupyter Notebooks, making it easy to create and display maps in your data analysis workflow.
- The **`explore()`** method returns a folium.Map object, which can also be passed directly (as you do with ax in plot()). You can then use folium functionality directly on the resulting map.

### Let's plot the vegetation polygons and focal plot centers with folium

In [ ]:
import folium
mapObj = folium.Map(location=[-33.95, 23.52], zoom_start=7, control_scale=True)

folium.GeoJson(veg_polys_gdf,
               name="BioSCape Vegetation Polygons", 
               tooltip=folium.GeoJsonTooltip(fields=["BScpPID"]), 
               style_function=lambda x: {"fillOpacity": 0}).add_to(mapObj)

folium.GeoJson(vegpolys_cen_new, 
               name="BioSCape Polygon Centers", 
               marker=folium.Circle(radius=30, fill_color="red", fill_opacity=0.8, color="red", weight=1),
               color="red").add_to(mapObj)

# create ESRI satellite base map
#esri = 'https://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{z}/{y}/{x}'
#folium.TileLayer(tiles = esri, attr = 'Esri', name = 'Esri Satellite', overlay = False, control = True).add_to(mapObj)
folium.LayerControl().add_to(mapObj)
mapObj

## Now let's explore a subset of polygon files that are in the Cape Peninsula region

### We'll focus on 5 Cape polygons that have dominant vegetation families for the focal plots as listed below.

- T081 = Proteaceae
- T093 = Asteraceae
- T201 = Ericaceae
- T069 = Rosaceae
- T195 = Restionaceae

#### Start by selecting BScpPID = T081

In [ ]:
veg_polys_gdf.loc[veg_polys_gdf['BScpPID'] == 'T081']

In [ ]:
# Plot the T081 polygon and center together
T081poly = veg_polys_json_gdf.loc[veg_polys_json_gdf['BScpPID'] == 'T081']
T081cen = vegpolys_cen_new.loc[vegpolys_cen_new['BScpPID'] == 'T081']
fig, ax = plt.subplots(figsize=(7,7))
T081poly.plot(ax=ax)
T081cen.plot(ax=ax, color='red')

#### Let's assign variables of the latitude / longitude coordinates of the vegetation center plot in the T081 Polygon

In [ ]:
T081_point = vegpolys_cen_new.loc[vegpolys_cen_new['BScpPID'] == 'T081'].get_coordinates()
T081lon = T081_point.x.values[0]
T081lat = T081_point.y.values[0]
print('T081lon: ', T081lon)
print('T081lat: ', T081lat)

## Demonstrate how to discover the AVIRIS-NG scenes associated with plot locations.  

AVIRIS-NG scenes of of interest for the Dominant Family Polygons are:
- ang20231109t131923_019
- ang20231109t133124_003
- ang20231109t133124_013

### AVIRIS-NG Flight Lines

#### A geoJSON of the AVIRIS-NG Flight Line's Scenes are available from the JPL BioSCape Data Portal 
- This file has been exported and is available in the BioSCape hub.

![AVIRISFlightLines](images/AVNG_flightlinedownload.PNG)

In [ ]:
# read and plot the AVNG coverage file
AVNG_Coverage = gpd.read_file('data/ANG_Coverage.geojson', driver='GeoJSON')
AVNG_Coverage.plot()

In [ ]:
list(AVNG_Coverage.columns)

In [ ]:
# Let's drop some unnecessary columns that result in errors in Folium visualization
AVNG_scenes = AVNG_Coverage.drop(columns=['LOC Download', 'LOC ORT Download', 'OBS Download', 'OBS ORT Download', 'Quicklook Download', 'RFL Download', 'RFL UNC Download', 'end_time', 'start_time', 'style'])

#### Let's visualize the veg polygons, veg centers, and AVIRIS-NG Flight Lines

In [ ]:
import folium
mapObj = folium.Map(location=[-33.95, 23.52], zoom_start=7, control_scale=True)

folium.GeoJson(veg_polys_gdf, 
               name="BioSCape Vegetation Polygons", 
               tooltip=folium.GeoJsonTooltip(fields=["BScpPID"]), 
               style_function=lambda x: {"fillOpacity": 0}).add_to(mapObj)

folium.GeoJson(vegpolys_cen_new, 
               name="BioSCape Polygon Centers", 
               marker=folium.Circle(radius=10, fill_color="red", fill_opacity=0.4, color="red", weight=1),
               color="red").add_to(mapObj)

folium.GeoJson(AVNG_scenes, 
               name="AVIRIS-NG Coverage", 
               color="red", 
               tooltip=folium.GeoJsonTooltip(fields=["scid"]), 
               style_function=lambda x: {"fillOpacity": 0}).add_to(mapObj)


# create ESRI satellite base map
#esri = 'https://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{z}/{y}/{x}'
#folium.TileLayer(tiles = esri, attr = 'Esri', name = 'Esri Satellite', overlay = False, control = True).add_to(mapObj)
folium.LayerControl().add_to(mapObj)
mapObj

#### We know that there are many AVIRIS-NG scenes in the BioSCape Campaign data.
#### Let's discover which AVIRIS-NG scenes contain the T081 polygon 

In [ ]:
AVNG_flight_scenes_T081_gdf = gpd.sjoin(veg_polys_gdf.loc[veg_polys_gdf['BScpPID'] == 'T081'], AVNG_scenes, how='inner', predicate='within')
AVNG_flight_scenes_T081_gdf.scid	

In [ ]:
# As an example, here's the same method for the T195 vegetation polygon
AVNG_flight_scenes_T195_gdf = gpd.sjoin(veg_polys_gdf.loc[veg_polys_gdf['BScpPID'] == 'T195'], AVNG_scenes, how='inner', predicate='within')
AVNG_flight_scenes_T195_gdf.scid

In [ ]:
AVNG_flight_scenes_T081_gdf

In [ ]:
# Lets use loc and the scid to plot just the ang20231109t131923_019
AVNG_scenes.loc[AVNG_scenes['scid'] == 'ang20231109t131923_019'].plot()

#### Now plot it all together with the identified AVIRIS-NG scene

In [ ]:
# and now plot it all together with the identified AVIRIS-NG scene
import folium
mapObj = folium.Map(location=[-33.95, 23.52], zoom_start=7, control_scale=True)

folium.GeoJson(veg_polys_gdf, 
               name="BioSCape Vegetation Polygons", 
               tooltip=folium.GeoJsonTooltip(fields=["BScpPID"]), 
               style_function=lambda x: {"fillOpacity": 0}).add_to(mapObj)

folium.GeoJson(vegpolys_cen_new, 
               name="BioSCape Polygon Centers", 
               marker=folium.Circle(radius=10, fill_color="red", fill_opacity=0.4, color="red", weight=1),
               color="red").add_to(mapObj)

folium.GeoJson(AVNG_scenes, 
               name="AVIRIS-NG Coverage", 
               color="red", 
               tooltip=folium.GeoJsonTooltip(fields=["scid"]), 
               style_function=lambda x: {"fillOpacity": 0}).add_to(mapObj)

folium.GeoJson(AVNG_scenes.loc[AVNG_scenes['scid'] == 'ang20231109t131923_019'], 
               name="AVIRIS-NG Coverage", 
               color="green", 
               tooltip=folium.GeoJsonTooltip(fields=["scid"]), 
               style_function=lambda x: {"fillOpacity": 0}).add_to(mapObj)

# create ESRI satellite base map
esri = 'https://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{z}/{y}/{x}'
folium.TileLayer(tiles = esri, attr = 'Esri', name = 'Esri Satellite', overlay = False, control = True).add_to(mapObj)
folium.LayerControl().add_to(mapObj)
mapObj

## Examine BioSCape AVIRIS-NG Files that have Vegetation Plot of Interest

### The ang20231109t131923_019 scene is a segment of a larger flight line, ang20231109t131923

In [ ]:
s3.ls('bioscape-data/AVNG/ang20231109t131923')

#### ^^^ There are 22 scenes in this (ang20231109t131923) flight line ^^^  

In [ ]:
# List the data files in the ang20231109t131923_019 scene

ang20231109t131923_019 =  s3.ls('bioscape-data/AVNG/ang20231109t131923/ang20231109t131923_019')
ang20231109t131923_019


- Flight lines are provided as small, manageable sections of data referred to as "scenes"
- Within a flight line, adjacent scenes are seamless
- adjacent flight lines will have overlap
- To date, data are in binary/header (ENVI) formats

| Dataset | Description | 
| -------- | --- |
| L1B | Resampled calibrated data in units of spectral radiance.  Observed geometry and illumination parameters |
| L2 | Orthocorrected and atmospherically corrected reflectance data. Uncertainty data.  (425 Spectral bands)|
| QL | A quick look geoTIFF file (3 bands)

## Visualize AVIRIS-NG Reflectance Files

### Examine the Quicklook file with GDAL

In [ ]:
# Define the path for a quick look image
ang20231109t131923_019_ql_path = [x for x in ang20231109t131923_019 if x.endswith('RFL_ORT_QL.tif')][0]
ang20231109t131923_019_ql_path

**There's a bit of a catch with the cloud access methods using GDAL.  The GDAL utility expects S3 links to be formated with the `GDAL virtual file system (VSI)` S3 path.  We therefore have to use the VSI path to access the files with GDAL.  What that means is shown below.  We'll substitute the S3 link with the VSI (`vsis3`) link(s).**

In [ ]:
ang20231109t131923_019_ql = gdal.Open(path.join('/vsis3', ang20231109t131923_019_ql_path))
ang20231109t131923_019_ql

In [ ]:
# Open the AVIRIS-NG ENVI file and read the file's bands, row, cols
#image_open = gdal.Open(gdal_url)

nbands_003 = ang20231109t131923_019_ql.RasterCount
ncols_003 = ang20231109t131923_019_ql.RasterXSize
nrows_003 = ang20231109t131923_019_ql.RasterYSize

print(f"Bands:\t{nbands_003}")
print(f"Columns:\t{ncols_003}")
print(f"Rows:\t{nrows_003}")

### Using GDAL to read the AVIRIS-NG raster file as a numerical array
- convert an existing **`Gdal Dataset`** or a Band into a numpy array with the `ReadAsArray()` function

In [ ]:
ang_019_ql_array = ang20231109t131923_019_ql.ReadAsArray()
ang_019_ql_trans = ang_019_ql_array.transpose((1,2,0))  # the band arranged needs rearranged in order to plot

In [ ]:
# bands 258 95 36
plt.rcParams['figure.figsize'] = [10,10]
plt.imshow(ang_019_ql_trans)

In [ ]:
## Quick note - you can use this get expression to download data from S3 storage to your local environment
## We will not execute this block in the workshop.

# s3.get(ang20231109t131923_019_ql_path, 'ang20231109t131923_019_ql.tif')

## Examine the AVNG Reflectance file (RFL) with rioxarray

**`Rioxarray`** is a python package that is built upon `xarray` and `rasterio` python packages to facilitate the analysis of raster or xarray datasest. 

We'll demonstrate 
- how to read and visualize raster data using rioxarray
- how to extract data from a point location.

### First, let's examine the header file associated with the ENVI binary/header file pair
- the header files are ascii text files

In [ ]:
hdr_link = 'bioscape-data/AVNG/ang20231109t131923/ang20231109t131923_019/ang20231109t131923_019_L2A_OE_main_27577724_RFL_ORT.hdr'
hdr_link

In [ ]:
# Print the whole header file
#with s3.open(hdr_link, mode='r') as hdr:
#    lines = (hdr.read())
#    print(lines)

# Print the first 12 lines of the header file
with s3.open(hdr_link, mode='r') as hdr:
    for i in range(12):
        line = next(hdr).strip()
        print(line) 

In [ ]:
# assign the the associated L2 Reflectance file to a variable
ang20231109t131923_019_rfl = [x for x in ang20231109t131923_019 if x.endswith('RFL_ORT')][0]
ang20231109t131923_019_rfl

In [ ]:
# open the reflectance file with rioxarray
avng_rfl = rioxarray.open_rasterio(path.join('s3://', ang20231109t131923_019_rfl),
                             drivers='ENVI', chunks=True, parse_coordinates=True)
avng_rfl

In [ ]:
# plot rgb bands - this takes a minute ...
avng_rfl.isel(band=[57, 38, 22]).plot.imshow(rgb="band", vmin=0, vmax=.15)
plt.title(path.basename(ang20231109t131923_019_rfl))
plt.show()

#### Notice that the x and y listed coordinates are in projected unit (meters)

In [ ]:
# Examine the coordinate reference system of the AVIRIS-NG file
avng_rfl.rio.crs

**The EPSG code is 32734**

In [ ]:
# View the Coordinate Reference System (CRS) & spatial extent
# explore the full definition of the EPSG code
from pyproj import CRS
epsg = avng_rfl.rio.crs.to_epsg()
crs = CRS(epsg)
crs

#### This AVIRIS-NG scence is in UTM Zone 34S projection

## Plot a Spectral Profile from a Point of Interest

### We'll plot the spectral profiles for a pixel from the vegeation plot location.

- Recall we saved the latitude and longitude of the T081 vegetation plot center.
- Project point location lat/lon to UTM (zone 34S): Recall, the AVIRIS-NG Scene is in UTM zone 34S (EPSG:32734)

In [ ]:
# recall
print('Plot_T081 longitude: ', T081lon)  
print('Plot_T081 latitude: ', T081lat)

#### Convert lon/lat to UTM projected x/y

**`Pyproj`** is a Python interface to PROJ (cartographic projections and coordinate transformations library)

In [ ]:
# translate coordinates

p = Proj("EPSG:32734", preserve_units=False)
T081_x,T081_y = p(T081lon, T081lat)
print('T081 x value in UTM meters:', T081_x)
print('T081 y value in UTM meters:', T081_y)

### Plot the Reflectance with the T081 plot location

In [ ]:
# Plot the image with the T081 plot location

plt.figure(figsize=(10,10))
avng_rfl.isel(band=[57, 38, 22]).plot.imshow(rgb="band", vmin=0, vmax=.25)
plt.scatter(T081_x,T081_y, color='red')
plt.title(path.basename(ang20231109t131923_019_rfl))
plt.show()


## For the  `T081` vegetation plot center location, let's extract the reflectance value from each of the 425 bands, and plot this as a spectral profile

In [ ]:
# plot spectra from the y,x pixel defined in UTM coordinates
T081_ang_wl = avng_rfl.sel(y=T081_y, x=T081_x, method='nearest')

T081_ang_wl.plot.line(ylim=(-.2,.4), color = 'g')
plt.rcParams['figure.figsize'] = [8,6]
plt.xlabel("band")
plt.ylabel("reflectance")
plt.show()


In [ ]:
# Define a list of wavelengths that are "bad" 
import numpy as np
bblist = np.ones((425,))  # create a 1D array with values ones
# set tails and atmospheric window to zero
bblist[0:14] = 0        # tail
bblist[193:210] = 0     # atmospheric window
bblist[281:320] = 0     # atmospheric window
bblist[405:] = 0        # tail

In [ ]:
T081_ang_wl[bblist == 0] = np.nan

plt.rcParams['figure.figsize'] = [8,6]
T081_ang_wl.plot.line(ylim=(0,.3), color = 'g')  
plt.xlabel("band")
plt.ylabel("reflectance")
plt.legend(["Proteaceae"])
plt.show()

## Plot the Spectral Profiles from each of Sample plots

AVIRIS-NG scenes of of interest for the Dominant Family Polygons are:

- T081 = Proteaceae ------ AVNG = ang20231109t131923_019_rfl
- T093 = Asteraceae ------ AVNG = ang20231109t133124_003_rfl
- T201 = Ericaceae -------- AVNG = ang20231109t133124_013_rfl
- T069 = Rosaceae -------- AVNG = ang20231109t133124_013_rfl
- T195 = Restionaceae ---- AVNG = ang20231109t133124_013_rfl


In [ ]:

# T081 from above

T093_point = vegpolys_cen_new.loc[vegpolys_cen_new['BScpPID'] == 'T093'].get_coordinates()
T093lon = T093_point.x.values[0]
T093lat = T093_point.y.values[0]

T201_point = vegpolys_cen_new.loc[vegpolys_cen_new['BScpPID'] == 'T201'].get_coordinates()
T201lon = T201_point.x.values[0]
T201lat = T201_point.y.values[0]

T069_point = vegpolys_cen_new.loc[vegpolys_cen_new['BScpPID'] == 'T069'].get_coordinates()
T069lon = T069_point.x.values[0]
T069lat = T069_point.y.values[0]

T195_point = vegpolys_cen_new.loc[vegpolys_cen_new['BScpPID'] == 'T195'].get_coordinates()
T195lon = T195_point.x.values[0]
T195lat = T195_point.y.values[0]

# Reproject point from lon/Lat to UTM
p = Proj("EPSG:32734", preserve_units=False)
T093_x,T093_y = p(T093lon, T093lat)
T201_x,T201_y = p(T201lon, T201lat)
T069_x,T069_y = p(T069lon, T069lat)
T195_x,T195_y = p(T195lon, T195lat)

avng_rfl_081_path = 'bioscape-data/AVNG/ang20231109t131923/ang20231109t131923_019/ang20231109t131923_019_L2A_OE_main_27577724_RFL_ORT'
avng_rfl_093_path = 'bioscape-data/AVNG/ang20231109t133124/ang20231109t133124_003/ang20231109t133124_003_L2A_OE_main_27577724_RFL_ORT'
avng_rfl_201_path = 'bioscape-data/AVNG/ang20231109t133124/ang20231109t133124_013/ang20231109t133124_013_L2A_OE_main_27577724_RFL_ORT'

# Read the reflectance file

#avng_rfl_081= rioxarray.open_rasterio(path.join('s3://', avng_rfl_081_path), drivers='ENVI', chunks=True, parse_coordinates=True)
avng_rfl_093 = rioxarray.open_rasterio(path.join('s3://', avng_rfl_093_path), drivers='ENVI', chunks=True, parse_coordinates=True)
avng_rfl_201 = rioxarray.open_rasterio(path.join('s3://', avng_rfl_201_path), drivers='ENVI', chunks=True, parse_coordinates=True)

T081_ang_wl[bblist == 0] = np.nan
T093_ang_wl = avng_rfl_093.sel(y=T093_y, x=T093_x, method='nearest')
T093_ang_wl[bblist == 0] = np.nan
T201_ang_wl = avng_rfl_201.sel(y=T201_y, x=T201_x, method='nearest')
T201_ang_wl[bblist == 0] = np.nan
T069_ang_wl = avng_rfl_201.sel(y=T069_y, x=T069_x, method='nearest')
T069_ang_wl[bblist == 0] = np.nan
T195_ang_wl = avng_rfl_201.sel(y=T195_y, x=T195_x, method='nearest')
T195_ang_wl[bblist == 0] = np.nan

# plot spectra from a pixel

T081_ang_wl.plot.line(ylim=(0,.4), color = 'g', label="Proteaceae")
T093_ang_wl.plot.line(ylim=(0,.4),color = 'r', label="Asteraceae")
T201_ang_wl.plot.line(ylim=(0,.4),color = 'y', label="Ericaceae")
T069_ang_wl.plot.line(ylim=(0,.4),color = 'purple', label="Rosaceae")
T195_ang_wl.plot.line(ylim=(0,.4),color = 'black', label="Restionaceae")

plt.rcParams['figure.figsize'] = [8,6]
plt.xlabel('band', fontsize=20)
plt.ylabel('Reflectance', fontsize=20)
plt.legend(loc="upper left")
plt.show()